In [4]:
import cv2
import json
import numpy as np
from deepface import DeepFace
from ultralytics import YOLO
import os

# pip install ultralytics deepface opencv-python numpy


model = YOLO("yolov8n-face.pt")

def register_student(name, image_path):
    # Detect face
    img = cv2.imread(image_path)
    results = model(img)[0]
    
    if len(results.boxes) == 0:
        print("No face detected!")
        return
    
    # Crop face
    box = results.boxes[0].xyxy[0]
    x1, y1, x2, y2 = map(int, box)
    face = img[y1:y2, x1:x2]

    # Save cropped face
    save_path = f"registered_faces/{name}.jpg"
    cv2.imwrite(save_path, face)

    # Extract embedding
    embedding = DeepFace.represent(face, model_name="Facenet")[0]["embedding"]

    # Save embedding in JSON
    db = {}
    if os.path.exists("embeddings.json"):
        db = json.load(open("embeddings.json"))

    db[name] = embedding
    json.dump(db, open("embeddings.json", "w"))

    print(f"{name} registered successfully!")


ModuleNotFoundError: No module named 'tensorflow.keras'

In [5]:
import cv2
import json
import numpy as np
from deepface import DeepFace
from ultralytics import YOLO

model = YOLO("yolov8n-face.pt")

# Load saved embeddings
db = json.load(open("embeddings.json"))

def find_best_match(face):
    # extract embedding
    emb = DeepFace.represent(face, model_name="Facenet")[0]["embedding"]

    min_dist = 1e9
    name = "Unknown"

    for student, db_emb in db.items():
        dist = np.linalg.norm(np.array(db_emb) - np.array(emb))

        if dist < min_dist and dist < 10:  # threshold
            min_dist = dist
            name = student

    return name

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    results = model(frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        face = frame[y1:y2, x1:x2]

        name = find_best_match(face)

        # Draw box and name
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, name, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("Attendance Camera", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'tensorflow.keras'

In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])


Predicted class: Egyptian cat


In [2]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


preprocessor_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/243M [00:00<?, ?B/s]

Detected cat with confidence 0.998 at location [344.06, 24.85, 640.34, 373.74]
Detected remote with confidence 0.997 at location [328.13, 75.93, 372.81, 187.66]
Detected remote with confidence 0.997 at location [39.34, 70.13, 175.56, 118.78]
Detected cat with confidence 0.998 at location [15.36, 51.75, 316.89, 471.16]
Detected couch with confidence 0.995 at location [-0.19, 0.71, 639.73, 474.17]
